In [ ]:
# for adding the videos to DB
# don't use at the same time with the server running
# https://stackoverflow.com/questions/59119396/how-to-use-django-3-0-orm-in-a-jupyter-notebook-without-triggering-the-async-con
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
#from backend.ml_models import DatabasePreferenceLearner
import numpy as np

from backend.models import Video, UserPreferences, DjangoUser, VideoRating
from backend.rating_fields import VIDEO_FIELDS

#%pylab
%matplotlib inline
#%matplotlib widget

from matplotlib import pyplot as plt
import mplcursors

import sys
import requests
from PIL import Image
import seaborn as sns
import pandas as pd

import datetime
from uuid import uuid1
import shortuuid

In [ ]:
import numpy as np

## Useful functions to generate fictitious data

# Takes any real z and outputs a number between 0 and 1
def logisticFunction(z):
    return 1/(1+np.exp(-z))

# This assumes that if video1 > video2, then the rating is closer to rating_min
def fullScore(user, video1, video2, scores, rating_min, rating_max):
    score_difference = scores[user][video1] - scores[user][video2]
    if (np.random.uniform() < logisticFunction(score_difference)):
        return rating_min
    return rating_max

# This assumes that if video1 > video2, then the rating is closer to rating_min
def intermediateScore(user, video1, video2, scores, rating_min, rating_max, intermediate_score_noise):
    score_difference = scores[user][video1] - scores[user][video2]
    rating_between_zero_one = 1-logisticFunction(score_difference)
    rating = (rating_max - rating_min) * rating_between_zero_one + rating_min
    noisy_rating = rating + intermediate_score_noise * (rating_max - rating_min) * np.random.normal()
    return min(rating_max, max(noisy_rating,rating_min))

def video_pair_recursive(video_pair, nb_remaining):
    if video_pair < nb_remaining - 1:
        return (video_pair, nb_remaining-1)
    (video1_temp, video2_temp) = video_pair_recursive(1+video_pair - nb_remaining, nb_remaining-1)
    return (video1_temp, video2_temp)

def video_pair_to_video1_video2(video_pair, nb_videos):
    if video_pair >= nb_videos * (nb_videos-1) /2:
        return (-1,-1) # Error
    return video_pair_recursive(video_pair, nb_videos)

def random_video1_video2(nb_videos):
    return video_pair_to_video1_video2(np.random.randint(nb_videos * (nb_videos-1)/2), nb_videos)

def random_user(nb_users, user_skewness):
    return int(nb_users * (np.random.uniform() ** (user_skewness)))

In [ ]:
# Assuming true_scores and learned_scores have as many users
def individual_scores_divergence(true_scores, learned_scores):
    sum_of_squares = 0
    nb_comparisons = 0
    for user in len(true_scores):
        for video in len(true_scores[0]):
            if learned_scores[user].has_key(video):
                sum_of_squares = sum_of_squares + (learned_scores[user][video] - true_scores[user][video])**2
                nb_comparisons = nb_comparisons + 1
    return np.sqrt(sum_of_squares / nb_comparisons)
    
def median_score(true_scores, video):
    list_scores = []
    for user in range(len(true_scores)):
        list_scores.append(true_scores[user][video])
    return np.median(list_scores)
    
# Assuming true_scores and learned_scores have as many users
def aggregate_scores_divergence(true_scores, tournesol_scores):
    sum_of_squares = 0
    for video in len(tournesol_scores):
        sum_of_squares = sum_of_squares + (median_score(true_scores,video) - tournesol_scores[video])**2
    return np.sqrt(sum_of_squares / len(tournesol_scores))
    


In [ ]:
## Parameters of the test dataset
nb_users = 10
nb_videos = 100

# Larger skewness means some users rated a lot more videos than others
# Skewness = 1 means that all users contributed equally
user_skewness = 2                  

# Sparsity of user inputs: smaller sparsity makes learning trickier
sparsity = .5

# Variance between the true scores of different users for the same video
score_variance_per_user = 2

# This assumes that the ratings range between 0 and rating_max
rating_min, rating_max = 0, 100    

# These are the minimum and maximum scores   
score_min, score_max = 1, 5        

# Users may provide min/max ratings, called full scores, or intermediate ratings
prob_fullScore = .5

# Larger intermediate score noises makes data noisier, and learning harder
intermediate_score_noise = .05

# This generates intrinsic scores for the videos
video_scores = np.random.uniform(score_min, score_max, nb_videos)

    
nb_datapoints_max = nb_users * nb_videos * (nb_videos-1) / 2
nb_datapoints = int(sparsity * nb_datapoints_max)

def generate_dataset():
    dataset = {}

    # This computes the users' scores for the videos that the ML model must partially recover
    scores = []
    for user in range(nb_users):
        scores.append(video_scores + score_variance_per_user * np.random.normal())


    for i in range(nb_datapoints):
        user = random_user(nb_users, user_skewness)
        video1, video2 = random_video1_video2(nb_videos)
        score_diff = scores[user][video1] - scores[user][video2]
        if (np.random.uniform() < prob_fullScore):
            dataset[user,video1,video2] = fullScore(user, video1, video2, scores, rating_min, rating_max)
        else: 
            dataset[user,video1,video2] = intermediateScore(user, video1, video2, scores, rating_min, rating_max, intermediate_score_noise)

    return {'dataset': dataset, 'scores': scores}

gen = generate_dataset()

In [ ]:
len(gen['dataset']), len(gen['scores'])

In [ ]:
plt.hist(np.array(list(gen['dataset'].values())))

In [ ]:
# clear everything
UserPreferences.objects.all().delete()
Video.objects.all().delete()

In [ ]:
prefix = shortuuid.ShortUUID().random(length=10)

In [ ]:
f = VIDEO_FIELDS[0]
f_default = {k: 50 for k in VIDEO_FIELDS if k != f}

In [ ]:
videos = []
for v in range(nb_videos):
    v = Video.objects.create(video_id=prefix + str(v), name="test video",
                         views=10, duration=datetime.timedelta(seconds=5), publication_date=datetime.datetime.now())
    videos.append(v)

In [ ]:
users = []
for u in range(nb_users):
    u = DjangoUser.objects.create(username=f"{prefix}_test_user__{u}", email=f"{u}@tournesol.com")
    users.append(u)
users = [UserPreferences.objects.create(user=u) for u in users]

In [ ]:
ratings = []
for (u, v1, v2), s in gen['dataset'].items():
    r = ExpertRating(user=users[u], video_1=videos[v1], video_2=videos[v2],
                    **f_default, **{f: s})
    ratings.append(r)
ratings = ExpertRating.objects.bulk_create(ratings)

In [ ]:
from django_react.settings import load_gin_config
from backend.management.commands.ml_train import learner

In [ ]:
load_gin_config('backend/ml_model/config/featureless_config.gin')

In [ ]:
learner_obj = learner()()
learner_obj.fit()
learner_obj.update_features()

In [ ]:
learner_obj.aggregator.plot_loss()

In [ ]:
gen['scores'][0]

In [ ]:
scores_out = [[getattr(VideoRating.objects.get(user=u, video=v), f) for v in videos] for u in users]

In [ ]:
scores_out_aggr = [getattr(Video.objects.get(id=v.id), f) for v in videos]

In [ ]:
plt.figure(figsize=(15, 6))
plt.title('Dataset and predicted scores')
for u in range(len(users)):
    plt.scatter(gen['scores'][u], scores_out[u], label=f'User {u}')
plt.xlabel('Dataset scores')
plt.ylabel('Predicted scores')
plt.legend()

In [ ]:
# Tournesol scores learned by the model are in average less than .5 from the true scores of the videos.

In [ ]:
def rank_loss(a, b):
    """ For given a, b compute the average number of misordered pairs, O(n^2) """
    
    # flattening data
    a, b = np.array(a).flatten(), np.array(b).flatten()
    
    # checking shape
    assert len(a) == len(b), "Lengths must agree"
    
    # sorting b in order of a
    b = np.array(b)[np.argsort(a)]
    
    # number of bad pairs
    res = sum([sum([1 if i < j and x >= y else 0 for j, y in enumerate(b)]) for i, x in enumerate(b)])
    
    # total number of pairs
    NN = len(a) * (len(a) - 1) / 2
    
    # return the ratio
    return 1. * res / NN

In [ ]:
for u in range(len(users)):
    rl = 100 * rank_loss(gen['scores'][u], scores_out[u])
    print(f"User {u} rank loss {round(rl, 2)}%")

In [ ]:
plt.figure(figsize=(15, 6))
plt.title('Dataset and predicted scores')
plt.scatter(video_scores, scores_out_aggr, label=f'Aggregated')
plt.xlabel('Dataset scores')
plt.ylabel('Predicted scores')
plt.legend()

rl = 100 * rank_loss(video_scores, scores_out_aggr)
print(f"User {u} rank loss {round(rl, 2)}%")